# Imports

In [ ]:
# pdf scanner
import PyPDF2
from pypdf import PdfReader
import pdfminer
# arrays
import pandas as pd
import numpy as np
# os
import os
# NLP
    # regex
import re
    # unicode
import unidecode

## ✅ Pulling Images (only required once)

In [ ]:
# reader object
# reader = PdfReader('data.pdf')

In [ ]:
# number of pages within data pdf
# num_pages = len(reader.pages)

In [ ]:
# images pulled from pdf
# reader = PdfReader("data.pdf")
# for page in reader.pages:
#     for image in page.images:
#         with open(image.name, "wb") as fp:
#             fp.write(image.data)

# Retrieving data from all pdfs

## Not using isolated pdf version of retrieval

### Proceeding with compiled PDF, code remains for documentation purposes

In [ ]:
# using pdfs seperated for \n inside the text

# importing required modules
# bodies = []
# names = ['dali','warhol','banksy','marcchagall','picasso','rembrandt','kaws','leoneardtsuguharufoujita','yayoi-kusama','zhangdaqian']
# # creating a pdf file object
# for name in names:
#     open_name = './' + name + '.pdf'
#     pdfFileObj = open(open_name, 'rb')
#     # creating a pdf reader object
#     pdfReader = PyPDF2.PdfReader(pdfFileObj)
#     # printing number of pages in pdf file
#     num_pages = len(pdfReader.pages)
#     print(f"Number of pages: {num_pages}")
#     # loop through all pages and extract text
#     for page_num in range(num_pages):
#         pageObj = pdfReader.pages[page_num]
#         text = pageObj.extract_text()
#         bodies.append(text)
#         print(f"Page {page_num+1}:\n{name}")
#         # validate the process body of text is correct format
#         if type(text) is str:
#             print('Successfully pulled text from page.\n')
#         else:
#             print('Failed to pull text.')
#     # closing the pdf file object
#     pdfFileObj.close()
#     print(f'+++++++++++++++++++++\nFinished text from {name} pdf\n+++++++++++++++++++++\n')


In [ ]:
# using as a whole pdf because it removes the line breaks \n

# importing required modules
bodies = []

# creating a pdf file object
pdfFileObj = open('./data.pdf', 'rb')

# creating a pdf reader object
pdfReader = PyPDF2.PdfReader(pdfFileObj)

# printing number of pages in pdf file
num_pages = len(pdfReader.pages)
print(f"Number of pages: {num_pages}")

    # loop through all pages and extract text
for page_num in range(num_pages):
    pageObj = pdfReader.pages[page_num]
    text = pageObj.extract_text()
    bodies.append(text)
    print(f"Page {page_num+1}:\ndata.pdf")
    # validate the process body of text is correct format
    if type(text) is str:
        print('Successfully pulled text from page.\n')
    else:
        print('Failed to pull text.')
# closing the pdf file object
pdfFileObj.close()


In [ ]:
# function for isolating and pulling jp & ch characters
def remove_ch_jp_characters(text):
  """
  Removes all chinese characters from a string
  """
  pattern = re.compile(r'[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\uff66-\uff9f]')
  return pattern.sub('', text)

In [ ]:
# decoding letters with accents into regular letters & removing character symbols
body_clean = []
for body in bodies:
    body = unidecode.unidecode(body)
    # removing the title of pdf from the body of text
    body = body.replace('Price Database31 October 2023','')
    body_clean.append(body)

In [ ]:
# separating each entry by artist name
names = ['Zhang Daqian', 'Andy Warhol', 'Banksy', 'Salvador Dali','Marc Chagell','Pablo Picasso',
         'Rembrandt van Rijn','KAWS','Leonard Tsuguharu Foujita','Yayoi Kusama']
# container for all entries
new_body = []
# container for each artist that has multiple names in the entry
banksy_clean = []
marc_clean = []
pablo_clean = []
rembrandt_clean = []
dali_clean = []
kaws_clean = []
for body in body_clean:
    for name in names:

        # isolating the entries that contain multiple counts of name in entry
            # the text in the if statement is the combined text where the name is repeated
            # these varie between title of piece or within the lot description


        # KAWS
        if name == 'KAWS':
            if 'Banksy and 21st Century Editions' in body:
                kaws_clean.append(body)
        else:
            pass
                
        # Banksy 
        if name == 'Banksy':
            if 'Only Banksy' in body:
                banksy_clean.append(body)
            elif 'Banksy and 21st Century Editions' in body:
                banksy_clean.append(body)
        else:
            pass

        # Marc Chagell
        if name == 'Marc Chagell':
            if 'Marc Chagall, Colour of Life' in body:
                marc_clean.append(body)
        # Dali
        if name == 'Salvador Dali':
            if 'After Salvador Dali' in body:
                dali_clean.append(body)
            elif 'SALVADOR DALI\' "Genesis"' in body:
                dali_clean.append(body)
            elif 'Currier & Ives as Interpreted by Salvador Dali' in body:
                dali_clean.append(body)
        else:
            pass
        # Picasso
        if name == 'Pablo Picasso':
            if 'After Pablo Picasso' in body:
                pablo_clean.append(body)
        # Rembrandt
        if name == 'Rembrandt van Rijn':
            if 'School of Rembrandt van Rijn' in body:
                rembrandt_clean.append(body)
            elif 'Circle of Rembrandt van Rijn' in body:
                rembrandt_clean.append(body)
            elif 'Studio of Rembrandt van Rijn' in body:
                rembrandt_clean.append(body)
            elif 'Follower of Rembrandt van Rijn' in body:
                rembrandt_clean.append(body)

        # All other values that do not have repeated names
        else:
            entries = body.split(f"{name}")[1:]
        for entry in entries:
            entry = entry.strip()
            # adding an additional space back in between the name and piece_name
            a = ''.join([name+' ',entry])
            a = a.replace('\x00','-')
            new_body.append(a)
            print(a)
            print("="*30)


In [ ]:
# Removing entries from new_body that do not have complete data
new_body = [body for body in new_body if len(body) >= 130]

## Isolated Lists of Entries that have the name of artist within the data.

### KAWS

In [ ]:
kaws_clean = list(set(kaws_clean))

In [ ]:
kaws_ = []
for i in kaws_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # isolating single that has banksy in the lot name
        if 'DISSECTEDCOMPANION' in entry:
        # adding space after artist name for ease of regex
            entry = entry.replace('KAWS','KAWS ')
            # removing double spaces
            entry = entry.replace('  ', ' ')
            kaws_.append(entry)
        else:
            pass

# cleaned KAWS entries with multiple appearances of names
kaws_

### Banksy

In [ ]:
# removing duplicates
banksy_clean = list(set(banksy_clean))

In [ ]:
banksy_ = []
for i in banksy_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('KAWS', 'KAWS ')
        entry = entry.replace('Banksy','Banksy ')
        if 'KAWS' in entry:
            pass
        else:
            # removing double spaces
            entry = entry.replace('  ', ' ')
            banksy_.append(entry)

# cleaned banksy entries with multiple appearances of names
banksy_

### Rembrandt

In [ ]:
# removing duplicates
rembrandt_clean = list(set(rembrandt_clean))

In [ ]:
rembrandt_ = []
for i in rembrandt_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Rembrandt van Rijn','Rembrandt van Rijn ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        rembrandt_.append(entry)

# cleaned rembrandt entries with multiple appearances of names
rembrandt_

### Marc Chagall

In [ ]:
marc_ = []
for i in marc_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Marc Chagall','Marc Chagall ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        marc_.append(entry)

# cleaned marc chagall entries with multiple appearances of names
marc_

### Pablo Picasso

In [ ]:
pablo_ = []
for i in pablo_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Pablo Picasso','Pablo Picasso ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        pablo_.append(entry)

# cleaned pablo picasso entries with multiple appearances of names
pablo_

### Salvador Dali

In [ ]:
dali_clean

In [ ]:
dali_ = []
for i in dali_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Dali','Dali ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        dali_.append(entry)

# cleaned dali entries with multiple appearances of names
dali_ = dali_[1:]
dali_

In [ ]:
# combing all cleaned entries
add_ins = dali_ + rembrandt_ + marc_ + pablo_ + banksy_ + kaws_
len(add_ins)

## Connecting multiple name counts with the rest of values that did not need extra cleaning

In [ ]:
print(len(new_body))

# new_body.extend(add_ins)
# Convert each item to a set of words
set_new_body = [set(item.split()) for item in new_body]
set_add_ins = [set(item.split()) for item in add_ins]

# Find items in new_body that are not similar to any item in add_ins
unique_items = [item for item, s in zip(new_body, set_new_body) if s not in set_add_ins]

# Merge unique items from new_body with add_ins
merged_list = unique_items + add_ins

print(len(new_body))

In [ ]:
len(add_ins)

In [ ]:
count_addins = {}
for artist in names:
    for body in add_ins:
        if artist in body:
            if artist in count_addins:
                count_addins[artist] += 1
            else:
                count_addins[artist] = 1

In [ ]:
pd.DataFrame([count_addins], index=['count']).T.sort_values(by='count',ascending=False)

In [ ]:
count_artist = {}
for artist in names:
    for body in new_body:
        if artist in body:
            if artist in count_artist:
                count_artist[artist] += 1
            else:
                count_artist[artist] = 1

In [ ]:
# Data Frame to see distribution of a
pd.DataFrame([count_artist], index=['count']).T.sort_values(by='count',ascending=False)

In [ ]:
new_body

In [ ]:
# reaffirming all cleaned entries are removed 
new_body = [body for body in new_body if len(body) >= 130]

## Isolating values by regex needs

In [ ]:
# pre regex
# 1. if name count < 1, drop value (DONE)

# regex
    # 2. Make depth optional
    # 3. make foreign currency (estimate & hammer price) optional
    # 4.

# vars
# new_body = collective of all values
# has_fc
    # fc_has_depth
    # fc_no_depth

# Step 1:
### Seperating by Foreign Currency

In [ ]:
# seperating by IF has foreign currency
    # list of all potential foreign currencies
foreign_currency = ['HKD','EUR','NZD','AUD','GBP','KRW','CNY','JPY','CHF','SGD','CAD','TWD','INR','THB','RUB','SEK','DKK','NOK',
                    'PLN','ZAR','MXN','BRL','TRY','AED','SAR','QAR','KWD','OMR','BHD','JOD','ILS','EGP','IDR','MYR','PHP','CZK',
                    'HUF','CLP','ARS','COP','PEN']
# seperating by IF has depth
# Depth as a keyword

In [ ]:
# new_body with foreign currency
has_foreign_currency = [body for body in new_body if any(currency in body for currency in foreign_currency)]

# new_body without foreign currency
no_foreign_currency = []
for body in new_body:
    if body not in has_foreign_currency:
        no_foreign_currency.append(body)


In [ ]:
print(f'Length of FC: {len(has_foreign_currency)},\
 Length of No FC: {len(no_foreign_currency)},\
 Combined = {len(has_foreign_currency) + len(no_foreign_currency)}')

In [ ]:
depth_fc = [body for body in has_foreign_currency if 'Depth' in body]

In [ ]:
len(depth_fc)

In [ ]:
depth_fc = depth_fc[1:]
depth_fc

In [ ]:
depth_has_estimate_fc = []
for string in depth_fc:
    match = re.search(r'\|', string)
    if match:
        depth_has_estimate_fc.append(string)
    else:
        pass

In [ ]:
print(len(depth_has_estimate_fc))
depth_has_estimate_fc

In [ ]:
for body in depth_has_estimate_fc:
    if '\n' in body:
        print(f'has n, \n{body}')

In [ ]:
import re

# Provided string
data = '''
'Banksy Napalmscreenprint in coloursHeight 50 x Width 70 cm.Height 19.685 x Width 27.559 in.2005Edition: 58/15013 December 202211 months agoBonhams LondonPrints and Multiples - [Lot 00085]est. 20,000 u 30,000 GBP25,500GBPest. 24,749 u 37,124 USD31,555USD',
'Banksy I Fought The LawscreenprintHeight 66 x Width 66.3 cm.Height 25.984 x Width 26.102 in.2005Edition: 50029 October 2022about 1 year agoSBI Art Auction Co., Ltd.The 54th SBI Art Auction|Modern andContemporary Art - [Lot 00298]est. 2,000,000 u 3,000,000 JPY2,127,500JPYest. 13,561 u 20,342 USD14,426USD',
"Rembrandt van Rijn Jan Lutma, Goldsmithetching with engraving and drypointHeight 19.7 x Width 14.9 cm.Height 7.756 x Width 5.866 in.165605 July 2022over 1 year agoChristie's LondonOld Master and British Works on Paper:Drawings, Watercolours and Prints 1500u1900- [Lot 00228]est. 15,000 u 25,000 GBP16,380GBPest. 17,906 u 29,843 USD19,553USD"
'''

# Define a regular expression pattern to extract information
pattern = r'''
    '([^']+)         # Match and capture the artist name in single quotes
    (.+?)Height       # Match any characters until 'Height'
    (\d+(\.\d+)?)     # Match and capture the height (with optional decimal)
    x Width           # Match 'x Width'
    (\d+(\.\d+)?)     # Match and capture the width (with optional decimal)
    in\.(\d{4})       # Match and capture the year
    Edition: (\d+/\d+) # Match and capture the edition
    (\d{1,2}\s\w+\sago) # Match and capture the time ago
    (.+?)est\.        # Match any characters until 'est.'
    (\d+,\d+)         # Match and capture the estimated price in GBP
    u (\d+,\d+)       # Match and capture the estimated price in USD
    GBP(\d+,\d+)      # Match and capture the final price in GBP
    USD(\d+,\d+)      # Match and capture the final price in USD
'''

matches = re.finditer(pattern, data, re.VERBOSE)

for match in matches:
    artist, title, height_cm, width_cm, year, edition, time_ago, \
    est_price_gbp, est_price_usd, final_price_gbp, final_price_usd = match.groups()

    print(f"Artist: {artist}")
    print(f"Title: {title}")
    print(f"Dimensions: Height {height_cm} x Width {width_cm} cm")
    print(f"Year: {year}")
    print(f"Edition: {edition}")
    print(f"Time Ago: {time_ago}")
    print(f"Estimated Price: GBP {est_price_gbp}, USD {est_price_usd}")
    print(f"Final Price: GBP {final_price_gbp}, USD {final_price_usd}")
    print("="*30)


In [ ]:
pd.DataFrame(matches)